In [4]:
import os
import logging
import joblib
import torch
from flask import Flask, request, jsonify
from transformers import AutoTokenizer

# =======================
#  Logging Setup
# =======================
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# =======================
#  Flask App Initialization
# =======================
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = True

# =======================
#  Model Loading Function
# =======================
def load_model(path: str, model_type="joblib"):
    """
    Loads a machine learning model from a given path.
    Args:
        path (str): Path to the model file.
        model_type (str): Type of model to load ('joblib' or 'transformer').
    Returns:
        The loaded model if successful, otherwise None.
    """
    try:
        logger.info(f"Loading model from {path}...")
        
        if model_type == "joblib":
            model = joblib.load(path)
        elif model_type == "transformer":
            model = AutoTokenizer.from_pretrained(path)
        else:
            raise ValueError(f"Unsupported model type: {model_type}")
        
        if hasattr(model, 'eval'):
            model.eval()  # Put model in evaluation mode if it's a neural network
        logger.info(f"✅ Successfully loaded model from {path}.")
        return model
    
    except FileNotFoundError:
        logger.error(f"❌ File not found: {path}. Please check the file path.")
    except ValueError as e:
        logger.error(f"❌ Value error while loading model from {path}: {e}")
    except Exception as e:
        logger.error(f"❌ Unexpected error while loading model from {path}: {e}")
    return None


# =======================
#  Model Loading Section
# =======================
try:
    models = {
        "data_class": load_model("./java_data_class", model_type="joblib"),
        "feature_envy": load_model("./java_feature_envy", model_type="joblib"),
        "long_parameter": load_model("./java_long_parameter", model_type="joblib"),
        "long_method": load_model("./java_long_method", model_type="joblib"),
        "javascript_graphcodebert": load_model("./javascript_graphcodebert", model_type="joblib"),
        "complex_conditional": load_model("./javascript_complex_conditional", model_type="joblib"),
        "java_blob": load_model("./java_blob", model_type="joblib"),
        "tokenizer": load_model("microsoft/graphcodebert-base", model_type="transformer")
    }

    for model_name, model in models.items():
        if model is None:
            logger.critical(f"❌ {model_name} model failed to load.")
            raise Exception(f"{model_name} model failed to load")
        else:
            logger.info(f"✅ {model_name} model loaded successfully.")

except Exception as e:
    logger.critical(f"❌ Critical error while loading models: {e}")
    raise e  # Stop the server from running if models are essential


# =======================
#  Flask Routes
# =======================
@app.route('/', methods=['GET'])
def home():
    """
    Root endpoint to check if the API is running.
    """
    return jsonify({
        'status': 'success',
        'message': 'Welcome to the Code Smell Detection API'
    }), 200


@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint to make predictions using one of the loaded models.
    Example request payload:
    {
        "model": "data_class",
        "input": "public class Test { private int x; }"
    }
    """
    try:
        # Input validation
        data = request.get_json()
        if not data or 'model' not in data or 'input' not in data:
            logger.warning("❌ Invalid input data")
            return jsonify({'error': 'Invalid request. Please provide "model" and "input" fields.'}), 400
        
        model_name = data['model']
        user_input = data['input']

        if model_name not in models:
            logger.warning(f"❌ Model '{model_name}' is not available.")
            return jsonify({'error': f'Model "{model_name}" not found. Available models: {list(models.keys())}'}), 404

        model = models.get(model_name)
        
        if not model:
            logger.warning(f"❌ Model '{model_name}' is not loaded.")
            return jsonify({'error': f'Model "{model_name}" is not loaded properly.'}), 500

        # Dummy logic for prediction (replace this with actual prediction logic)
        if model_name in ['data_class', 'feature_envy', 'long_parameter', 'long_method', 'complex_conditional', 'java_blob']:
            prediction = model.predict([user_input])  # Assuming the model has a 'predict' method
        elif model_name == 'javascript_graphcodebert':
            prediction = model.predict([user_input])  # Custom logic for the graphcodebert model
        elif model_name == 'tokenizer':
            prediction = model.encode(user_input)  # Tokenizer logic

        logger.info(f"✅ Prediction successful for model '{model_name}'")
        return jsonify({
            'model': model_name,
            'input': user_input,
            'prediction': prediction.tolist() if hasattr(prediction, 'tolist') else prediction
        }), 200

    except Exception as e:
        logger.error(f"❌ Error during prediction: {e}")
        return jsonify({'error': 'An internal error occurred during prediction.'}), 500


@app.errorhandler(404)
def page_not_found(e):
    """
    Custom 404 error handler.
    """
    return jsonify({'error': 'The requested URL was not found on the server.'}), 404


@app.errorhandler(500)
def internal_server_error(e):
    """
    Custom 500 error handler.
    """
    return jsonify({'error': 'An internal server error occurred.'}), 500


@app.errorhandler(Exception)
def handle_exception(e):
    """
    Handle all uncaught exceptions.
    """
    logger.error(f"❌ Unhandled Exception: {e}")
    return jsonify({'error': 'An unexpected error occurred.'}), 500


# =======================
#  Run Flask App
# =======================
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9005, debug=True)


INFO:__main__:Loading model from ./java_data_class...
INFO:__main__:✅ Successfully loaded model from ./java_data_class.
INFO:__main__:Loading model from ./java_feature_envy...
INFO:__main__:✅ Successfully loaded model from ./java_feature_envy.
INFO:__main__:Loading model from ./java_long_parameter...
INFO:__main__:✅ Successfully loaded model from ./java_long_parameter.
INFO:__main__:Loading model from ./java_long_method...
INFO:__main__:✅ Successfully loaded model from ./java_long_method.
INFO:__main__:Loading model from ./javascript_graphcodebert...
INFO:__main__:✅ Successfully loaded model from ./javascript_graphcodebert.
INFO:__main__:Loading model from ./javascript_complex_conditional...
INFO:__main__:✅ Successfully loaded model from ./javascript_complex_conditional.
INFO:__main__:Loading model from ./java_blob...
INFO:__main__:✅ Successfully loaded model from ./java_blob.
INFO:__main__:Loading model from microsoft/graphcodebert-base...
DEBUG:urllib3.connectionpool:Resetting droppe

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9005
 * Running on http://172.31.13.55:9005
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9005
 * Running on http://172.31.13.55:9005
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

In [ ]:
import esprima
from collections import defaultdict
from typing import List, Tuple



with open("demofile.js", "r", encoding="utf-8") as file:
    javascript_code = file.read()

unused_variables, unused_functions = find_unused_js_code(javascript_code)

print("Unused Variables:")
for var in unused_variables:
    print(f"- {var}")

print("\nUnused Functions:")
for func in unused_functions:
    print(f"- {func['name']}: {func['body']}")
